In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_text
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

%matplotlib inline

### Dataset

In this homework, we will use the California Housing Prices from [Kaggle](https://www.kaggle.com/datasets/camnugent/california-housing-prices).

Here's a wget-able [link](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv):

```bash
wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
```

The goal of this homework is to create a regression model for predicting housing prices (column `'median_house_value'`).

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


### Preparing the dataset 

For this homework, we only want to use a subset of data. This is the same subset we used in homework #2.
But in contrast to homework #2 we are going to use all columns of the dataset.

First, keep only the records where `ocean_proximity` is either `'<1H OCEAN'` or `'INLAND'`

Preparation:

* Fill missing values with zeros.
* Apply the log tranform to `median_house_value`.
* Do train/validation/test split with 60%/20%/20% distribution. 
* Use the `train_test_split` function and set the `random_state` parameter to 1.
* Use `DictVectorizer(sparse=True)` to turn the dataframes into matrices.

In [4]:
df = data.loc[data.ocean_proximity.isin(['<1H OCEAN', 'INLAND'])].reset_index(drop=True)

df.fillna(0, inplace=True)

df['median_house_value'] = np.log1p(df.median_house_value.to_numpy())

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.median_house_value.values
y_val = df_val.median_house_value.values
y_test = df_test.median_house_value.values

df_train.drop(['median_house_value'], axis=1, inplace=True)
df_val.drop(['median_house_value'], axis=1, inplace=True)
df_test.drop(['median_house_value'], axis=1, inplace=True)

dv = DictVectorizer(sparse=True)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)
val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

## Question 1

Let's train a decision tree regressor to predict the `median_house_value` variable. 

* Train a model with `max_depth=1`.


Which feature is used for splitting the data?

* `ocean_proximity`
* `total_rooms`
* `latitude`
* `population`

In [5]:
dt = DecisionTreeRegressor(max_depth=1)
dt.fit(X_train, y_train)

print(export_text(dt, feature_names=dv.feature_names_))

|--- ocean_proximity=<1H OCEAN <= 0.50
|   |--- value: [11.61]
|--- ocean_proximity=<1H OCEAN >  0.50
|   |--- value: [12.30]



## Question 2

Train a random forest model with these parameters:

* `n_estimators=10`
* `random_state=1`
* `n_jobs=-1` (optional - to make training faster)


What's the RMSE of this model on validation?

* 0.045
* 0.245
* 0.545
* 0.845

In [6]:
rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_val)
print(f"The RMSE of this model on the validation set is {round(np.sqrt(mean_squared_error(y_val, y_pred)), 3)}")

The RMSE of this model on the validation set is 0.245


## Question 3

Now let's experiment with the `n_estimators` parameter

* Try different values of this parameter from 10 to 200 with step 10.
* Set `random_state` to `1`.
* Evaluate the model on the validation dataset.


After which value of `n_estimators` does RMSE stop improving?

- 10
- 25
- 50
- 160

In [7]:
scores = []

for n in np.arange(10, 200, 10, dtype=int):
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train, y_train)
    
    y_pred = rf.predict(X_val)
    score = np.sqrt(mean_squared_error(y_val, y_pred))
    
    scores.append([n, round(score, 3)])

df_scores = pd.DataFrame(scores, columns=['n_estimators', 'rmse'])

df_scores["rmse_diff"] = df_scores["rmse"].diff()

best_n_estimators = int(df_scores.loc[df_scores.rmse_diff>=0, 'n_estimators'].head(1).values)

print(f"RMSE stops improving after {best_n_estimators} trees")

RMSE stops improving after 50 trees


/tmp/ipykernel_11185/553662491.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  best_n_estimators = int(df_scores.loc[df_scores.rmse_diff>=0, 'n_estimators'].head(1).values)


## Question 4

Let's select the best `max_depth`:

* Try different values of `max_depth`: `[10, 15, 20, 25]`
* For each of these values, try different values of `n_estimators` from 10 till 200 (with step 10)
* Fix the random seed: `random_state=1`


What's the best `max_depth`:

* 10
* 15
* 20
* 25

In [8]:
scores = []

for d in [10, 15, 20, 25]:
    
    for n in np.arange(10, 200, 10, dtype=int):
        rf = RandomForestRegressor(n_estimators=n,
                                   max_depth=d,
                                   random_state=1, 
                                   n_jobs=-1,
                                   warm_start=True)
        rf.fit(X_train, y_train)

        y_pred = rf.predict(X_val)
        score = np.sqrt(mean_squared_error(y_val, y_pred))

        scores.append((d, n, score))

df_scores = pd.DataFrame(scores, columns=['max_depth', 'n_estimators', 'rmse'])

min_score_max_depth = int(df_scores.groupby("max_depth", as_index=False)["rmse"] \
                                      .min() \
                                      .sort_values(by=['rmse'])['max_depth'] \
                                      .head(1) \
                                      .values)

mean_score_max_depth = int(df_scores.groupby("max_depth", as_index=False)["rmse"] \
                                      .mean() \
                                      .sort_values(by=['rmse'])['max_depth'] \
                                      .head(1) \
                                      .values)

print(f"Best max_depth considering RMSE min is {min_score_max_depth}")
print(f"Best max_depth considering RMSE mean is {mean_score_max_depth}")

Best max_depth considering RMSE min is 25
Best max_depth considering RMSE mean is 25


/tmp/ipykernel_11185/4119146460.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  min_score_max_depth = int(df_scores.groupby("max_depth", as_index=False)["rmse"] \
/tmp/ipykernel_11185/4119146460.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  mean_score_max_depth = int(df_scores.groupby("max_depth", as_index=False)["rmse"] \



## Question 5

We can extract feature importance information from tree-based models. 

At each step of the decision tree learning algorith, it finds the best split. 
When doint it, we can calculate "gain" - the reduction in impurity before and after the split. 
This gain is quite useful in understanding what are the imporatant features 
for tree-based models.

In Scikit-Learn, tree-based models contain this information in the
[`feature_importances_`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor.feature_importances_)
field. 

For this homework question, we'll find the most important feature:

* Train the model with these parametes:
    * `n_estimators=10`,
    * `max_depth=20`,
    * `random_state=1`,
    * `n_jobs=-1` (optional)
* Get the feature importance information from this model


What's the most important feature (among these 4)? 

* `total_rooms`
* `median_income`
* `total_bedrooms`
* `longitude`

In [9]:
rf = RandomForestRegressor(n_estimators=10, 
                           max_depth=20, 
                           random_state=1, 
                           n_jobs=-1)
rf.fit(X_train, y_train)

df_importances = pd.DataFrame()
df_importances['feature'] = dv.feature_names_
df_importances['importance'] = rf.feature_importances_

most_important_feature = df_importances.sort_values(by='importance', ascending=False)['feature'].head(1).values[0]

print(f"The most important feature is {most_important_feature}")

The most important feature is median_income


## Question 6

Now let's train an XGBoost model! For this question, we'll tune the `eta` parameter:

* Install XGBoost
* Create DMatrix for train and validation
* Create a watchlist
* Train a model with these parameters for 100 rounds:

```
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
```

Now change `eta` from `0.3` to `0.1`.

Which eta leads to the best RMSE score on the validation dataset?

* 0.3
* 0.1
* Both give equal value

In [10]:
features = dv.feature_names_

features = [col.replace("<", "lt") for col in features]

dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)

watchlist = [(dtrain, 'train'), (dval, 'val')]

scores = {}

In [11]:
progress = {}

xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,

    'objective': 'reg:squarederror',
    'nthread': 8,

    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=100,
                  evals=watchlist, verbose_eval=False, evals_result=progress)

In [12]:
scores['eta=0.3'] = pd.DataFrame(data={'num_iter': range(0, len(progress['train']['rmse'])),
                                       'train_rmse': progress['train']['rmse'],
                                       'val_rmse': progress['val']['rmse']})

In [13]:
progress = {}

xgb_params = {
    'eta': 0.1, 
    'max_depth': 6,
    'min_child_weight': 1,

    'objective': 'reg:squarederror',
    'nthread': 8,

    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=100,
                  evals=watchlist, verbose_eval=False, evals_result=progress)

In [14]:
scores['eta=0.1'] = pd.DataFrame(data={'num_iter': range(0, len(progress['train']['rmse'])),
                                       'train_rmse': progress['train']['rmse'],
                                       'val_rmse': progress['val']['rmse']})

In [15]:
if scores['eta=0.3'].val_rmse.min() < scores['eta=0.1'].val_rmse.min():
    print("eta=0.3 leads to the best RMSE score on the validation dataset")
elif scores['eta=0.3'].val_rmse.min() > scores['eta=0.1'].val_rmse.min():
    print("eta=0.1 leads to the best RMSE score on the validation dataset")
else:
    print("Both give equal value")

eta=0.3 leads to the best RMSE score on the validation dataset
